In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Load Data

In [2]:
# No rescaling needed. MNIST data is in range [0, 1]
mnist = input_data.read_data_sets("tf_data/")

Extracting tf_data/train-images-idx3-ubyte.gz
Extracting tf_data/train-labels-idx1-ubyte.gz
Extracting tf_data/t10k-images-idx3-ubyte.gz
Extracting tf_data/t10k-labels-idx1-ubyte.gz


## Model

In [68]:
# Discriminator Neural Network
class Discriminator(object):
    def __init__(self):
        """ Init the model with hyper-parameters etc. """
        # Layer 1 Variables
        self.weights1 = tf.Variable(tf.zeros([784, 1024]))
        self.biases1 = tf.Variable(tf.zeros([1024]))
        # Layer 2 Variables
        self.weights2 = tf.Variable(tf.zeros([1024, 512]))
        self.biases2 = tf.Variable(tf.zeros([512]))
        # Layer 3 Variables
        self.weights3 = tf.Variable(tf.zeros([512, 256]))
        self.biases3 = tf.Variable(tf.zeros([256]))
        # Out Layer Variables
        self.weights4 = tf.Variable(tf.zeros([256, 1]))
        self.biases4 = tf.Variable(tf.zeros([1]))
        # Store Variables in list
        self.var_list = [self.weights1, self.biases1, self.weights2, self.biases2, 
                           self.weights3, self.biases3, self.weights4, self.biases4]
            
    def inference(self, x):
        """ This is the forward calculation from x to y """
        x = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, self.weights1) + self.biases1), .3)
        x = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, self.weights2) + self.biases2), .3)
        x = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, self.weights3) + self.biases3), .3)
        x = tf.nn.softmax(tf.matmul(x, self.weights4) + self.biases4)
        return x
    
    def loss(self, real_data, noise, generator):
        fake_data = generator.inference(noise)
        y_fake = self.inference(fake_data)
        y_real = self.inference(real_data)
        return -tf.reduce_mean(tf.log(y_real) + tf.log(1. - y_fake), name='D_loss')
        
    def optimize(self, real_data, noise, generator):
        loss = self.loss(real_data, noise, generator)
        return tf.train.AdamOptimizer().minimize(loss, name='D_optimizer',
                                                 var_list=self.var_list)

In [69]:
# Geneator Neural Network
class Generator(object):
    def __init__(self):
        """ Init the model with hyper-parameters etc. """
        # Layer 1 Variables
        self.weights1 = tf.Variable(tf.zeros([100, 256]))
        self.biases1 = tf.Variable(tf.zeros([256]))
        # Layer 2 Variables
        self.weights2 = tf.Variable(tf.zeros([256, 512]))
        self.biases2 = tf.Variable(tf.zeros([512]))
        # Layer 3 Variables
        self.weights3 = tf.Variable(tf.zeros([512, 1024]))
        self.biases3 = tf.Variable(tf.zeros([1024]))
        # Out Layer Variables
        self.weights4 = tf.Variable(tf.zeros([1024, 784]))
        self.biases4 = tf.Variable(tf.zeros([784]))
        # Store Variables in list
        self.var_list = [self.weights1, self.biases1, self.weights2, self.biases2, 
                           self.weights3, self.biases3, self.weights4, self.biases4]
            
    def inference(self, x):
        """ This is the forward calculation from x to y """
        x = tf.nn.leaky_relu(tf.matmul(x, self.weights1) + self.biases1)
        x = tf.nn.leaky_relu(tf.matmul(x, self.weights2) + self.biases2)
        x = tf.nn.leaky_relu(tf.matmul(x, self.weights3) + self.biases3)
        x = tf.nn.tanh(tf.matmul(x, self.weights4) + self.biases4)
        return x
    
    def loss(self, noise, discriminator):
        fake_data = self.inference(noise)
        y_fake = discriminator.inference(fake_data)
        return -tf.reduce_mean(tf.log(y_fake), name='G_loss')
        
    def optimize(self, noise, discriminator):
        loss = self.loss(noise, discriminator)
        print(loss)
        return tf.train.AdamOptimizer().minimize(loss, name='G_optimizer',
                                                 var_list=self.var_list)

In [70]:
def noise(size):
    return tf.random_normal(shape=(size,100))

In [71]:
import numpy as np
a = np.expand_dims(np.arange(784),axis=0)
G_out = Generator().inference(tf.random_normal(shape=(1,100)))
D_out = Discriminator().inference(tf.convert_to_tensor(a, np.float32))

## Train

In [72]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
num_epochs = 100
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
summary = tf.summary.merge_all()
for _ in range(num_epochs):
    
    real_images, _ = mnist.train.next_batch(100)
    
    # Train Discriminator with real and fake data
    G_noise = noise(real_images.shape[0])
    opt = discriminator.optimize(real_images, G_noise, generator)
    sess.run(opt)
    summary_str = sess.run(summary, feed_dict=feed_dict)
    summary_writer.add_summary(summary_str, step)


    # Train Generator with fake data
    D_noise = noise(real_images.shape[0])
    generator.optimize(D_noise, discriminator)
#    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})